In [15]:
import pandas as pd
import numpy as np
import duckdb

In [16]:
temporadas=["2011/2012","2012/2013","2014/2015","2015/2016"]
carpeta = "./tablas_finales/"
equipo = pd.read_csv(carpeta+"equipo_españa.csv")
liga = pd.read_csv(carpeta+"liga.csv")
partido = pd.read_csv(carpeta+"partido_españa.csv", parse_dates=["Fecha"])
plantel = pd.read_csv(carpeta+"plantel.csv")
jugador = pd.read_csv(carpeta+"jugador.csv")
atributos = pd.read_csv(carpeta+"atributos.csv", parse_dates=["Fecha"])


In [17]:
# Selecciono partidos y plantel
partido_ok = partido[partido["Temporada"].isin(temporadas)]
plantel_ok = plantel[plantel["Temporada"].isin(temporadas)]

In [18]:
# Liga española
liga = 21518

¿cuales son los jugadores que más partido ganó su equipo?

In [19]:
partido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Id_Partido     2706 non-null   int64         
 1   Fecha          2706 non-null   datetime64[ns]
 2   Temporada      2706 non-null   object        
 3   Id_Local       2706 non-null   int64         
 4   Id_Visitante   2706 non-null   int64         
 5   Resultado      2706 non-null   object        
 6   Id_Liga        2706 non-null   int64         
 7   Gol_local      2706 non-null   int64         
 8   Gol_visitante  2706 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 190.4+ KB


In [20]:
partidos_ganados_local = duckdb.query("""
    SELECT Id_Local as Id_Equipo
    FROM partido_ok
    WHERE Resultado = 'Ganado'
""").df()

ganados_por_locales = duckdb.query("""
    SELECT Id_Equipo, COUNT(*) AS ganados_local
    FROM partidos_ganados_local
    GROUP BY Id_Equipo
    ORDER BY ganados_local DESC;
""").df()
ganados_por_locales["Id_Equipo"].is_unique

True

In [21]:
partidos_ganados_visitante = duckdb.query("""
    SELECT Id_Visitante as Id_Equipo
    FROM partido_ok
    WHERE Resultado = 'Perdido'
""").df()

ganados_por_visitantes = duckdb.query("""
    SELECT Id_Equipo, COUNT(*) AS ganados_visitante
    FROM partidos_ganados_visitante
    GROUP BY Id_Equipo
    ORDER BY ganados_visitante DESC;
""").df()

print(partidos_ganados_visitante.head())
print(ganados_por_visitantes.head())


   Id_Equipo
0       9906
1       8634
2       8305
3       8661
4       9869
   Id_Equipo  ganados_visitante
0       8634                 52
1       8633                 50
2       9906                 34
3      10267                 24
4       8315                 20


In [22]:
ganados_por_visitantes["Id_Equipo"].is_unique

True

In [23]:
## Chequeo que los índices sean iguales en locales y visitantes para hacer la suma
## Para eso utilizo la doble inclusión.
c1 = ganados_por_locales["Id_Equipo"].isin(ganados_por_visitantes["Id_Equipo"]).all()
c2 = ganados_por_visitantes["Id_Equipo"].isin(ganados_por_locales["Id_Equipo"]).all()

print(c1 & c2)

True


In [24]:
victorias_x_equipo = duckdb.query("""
    SELECT T1.Id_Equipo as Id_Equipo, ganados_local+ganados_visitante as ganados_totales 
    FROM ganados_por_locales as T1 INNER JOIN ganados_por_visitantes as T2 
    ON T1.Id_Equipo = T2.Id_Equipo
    ORDER BY ganados_totales DESC;
""").df()
victorias_x_equipo

,Id_Equipo,ganados_totales
0,8633,114
1,8634,114
2,9906,87
3,10267,69
4,8302,64
5,9864,58
6,8560,53
7,8315,53
8,8558,45
9,8581,44


In [25]:

duckdb.query("""
    SELECT Id_Equipo
    FROM victorias_x_equipo 
    WHERE ganados_totales = (SELECT MAX(ganados_totales) FROM victorias_x_equipo);
""")
# print(res)

┌───────────┐
│ Id_Equipo │
│   int64   │
├───────────┤
│      8633 │
│      8634 │
└───────────┘

¿Cual es el jugador con menor variación de potencia a lo largo de los años?

Analizo desde el 1° de Enero del 2011 al 31 de Diciembre del 2015

¿Qué busco cómo variación?

Por cada jugador obtengo el mínimo, y el máximo. Al máximo le resto el mínimo. Ahí tengo la variación a lo largo de los años 2011-2015

descarto jugadores que no hayan estado en las 4 temporadas

In [41]:
plantel_ok

,Id_Jugador,Id_Equipo,Temporada
0,95597,4087,2011/2012
1,38805,4087,2011/2012
2,35560,4087,2011/2012
3,210691,4087,2011/2012
4,93343,4087,2011/2012
...,...,...,...
19570,415113,208931,2015/2016
19571,105535,208931,2015/2016
19572,128803,208931,2015/2016
19573,24123,208931,2015/2016


In [56]:
plantel_ok.info()
jugadores_todas_temporadas = duckdb.query("""
    SELECT Id_Jugador
    FROM plantel_ok
    WHERE Temporada IN ('2011/2012', '2012/2013', '2013/2014', '2014/2015')
    GROUP BY Id_Jugador
    HAVING COUNT(DISTINCT Temporada) = 3;             
""")

potencial_ok = duckdb.query("""
    SELECT T1.Id_Jugador, Potencial, Fecha
    FROM jugadores_todas_temporadas as T1 LEFT JOIN atributos as T2
    ON T1.Id_Jugador = T2.Id_Jugador
""")

potencial_ok

<class 'pandas.core.frame.DataFrame'>
Index: 10847 entries, 0 to 19574
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Id_Jugador  10847 non-null  int64 
 1   Id_Equipo   10847 non-null  int64 
 2   Temporada   10847 non-null  object
dtypes: int64(2), object(1)
memory usage: 339.0+ KB


┌────────────┬───────────┬─────────────────────┐
│ Id_Jugador │ Potencial │        Fecha        │
│   int64    │  double   │    timestamp_ns     │
├────────────┼───────────┼─────────────────────┤
│      27671 │      77.0 │ 2014-11-28 00:00:00 │
│      27671 │      77.0 │ 2014-09-18 00:00:00 │
│      27671 │      80.0 │ 2014-04-25 00:00:00 │
│      27671 │      82.0 │ 2014-01-17 00:00:00 │
│      27671 │      83.0 │ 2013-12-20 00:00:00 │
│      27671 │      83.0 │ 2013-11-01 00:00:00 │
│      27671 │      83.0 │ 2013-10-18 00:00:00 │
│      27671 │      83.0 │ 2013-10-11 00:00:00 │
│      27671 │      83.0 │ 2013-09-27 00:00:00 │
│      27671 │      81.0 │ 2013-09-20 00:00:00 │
│        ·   │        ·  │          ·          │
│        ·   │        ·  │          ·          │
│        ·   │        ·  │          ·          │
│      51553 │      84.0 │ 2011-08-30 00:00:00 │
│      51553 │      83.0 │ 2011-02-22 00:00:00 │
│      51553 │      83.0 │ 2010-08-30 00:00:00 │
│      51553 │      

In [57]:
potencial_rango = duckdb.query("""
    SELECT * FROM potencial_ok
    WHERE Fecha between '2011-01-01' and '2016-01-01'
""" ).df()
potencial_rango

,Id_Jugador,Potencial,Fecha
0,27671,77.0,2014-11-28
1,27671,77.0,2014-09-18
2,27671,80.0,2014-04-25
3,27671,82.0,2014-01-17
4,27671,83.0,2013-12-20
...,...,...,...
20832,39549,77.0,2013-02-15
20833,39549,77.0,2012-08-31
20834,39549,77.0,2012-02-22
20835,39549,75.0,2011-08-30


Observación, hay jugadores 

In [61]:
res_raw = duckdb.query("""
    SELECT Id_Jugador, STDDEV(Potencial) AS Variacion_Potencial 
    FROM potencial_rango
    GROUP BY Id_Jugador
    ORDER BY Variacion_Potencial
    DESC;
""" ).df()
Jugador_Variacion_Potencial = duckdb.query("""
    SELECT * FROM res_raw WHERE Variacion_Potencial IS NOT NULL
""").df()

Jugador_Mayor_Variacion = duckdb.query("""
    SELECT * FROM Jugador_Variacion_Potencial
    WHERE Variacion_Potencial = (SELECT MAX(Variacion_Potencial) FROM Jugador_Variacion_Potencial)
""")

Jugador_Menor_Variacion = duckdb.query("""
    SELECT * FROM Jugador_Variacion_Potencial
    WHERE Variacion_Potencial = (SELECT MIN(Variacion_Potencial) FROM Jugador_Variacion_Potencial)
""")
print(Jugador_Mayor_Variacion)
print(Jugador_Menor_Variacion)

┌────────────┬─────────────────────┐
│ Id_Jugador │ Variacion_Potencial │
│   int64    │       double        │
├────────────┼─────────────────────┤
│      38994 │   6.166257152659728 │
└────────────┴─────────────────────┘

┌────────────┬─────────────────────┐
│ Id_Jugador │ Variacion_Potencial │
│   int64    │       double        │
├────────────┼─────────────────────┤
│     192316 │                 0.0 │
└────────────┴─────────────────────┘



In [65]:
Nombre_Menor_Variacion = duckdb.query("""
    SELECT Nombre FROM Jugador_Menor_Variacion as T1 INNER JOIN jugador as T2 on T1.Id_Jugador = T2.Id_Jugador
""")
Nombre_Mayor_Variacion = duckdb.query("""
    SELECT Nombre FROM Jugador_Mayor_Variacion as T1 INNER JOIN jugador as T2 on T1.Id_Jugador = T2.Id_Jugador
""")
print(Nombre_Menor_Variacion)
print(Nombre_Mayor_Variacion)

┌────────────────┐
│     Nombre     │
│    varchar     │
├────────────────┤
│ Mattias Autret │
└────────────────┘

┌─────────┐
│ Nombre  │
│ varchar │
├─────────┤
│ Rafael  │
└─────────┘

